In [55]:
import agentpy as ap
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import IPython

- Movimiento
- Recorrido
- Detenerse con obstáculo
- Respetar semáforo

Idea para el comportamiento general del carro:
- Primero revisará si las condiciones son adecuadas para moverse (Semáforos y otros carros)
- Si no se puede mover se espera
- Si se puede mover lo hace siguiendo una ruta designada desde el principio

In [56]:
class Carro(ap.Agent):

    def setup(self, x, y):
        
        #def __init__(self, grid, uniqueid, direction, rotation):
        self.grid = self.model.grid
        self.random = self.model.random
        self.group = 1
        #self.direction = direction
        #self.rotation = rotation
        
        # Posición inicial y cantidad de movimiento
        #arriba = (self.pos[0]-1, self.pos[1])
        #abajo = (self.pos[0]+1, self.pos[1])
        #izquierda = (self.pos[0], self.pos[1]-1)
        #derecha = (self.pos[0], self.pos[1]+1)

        self.stepAct = 0
        self.x = x
        self.y = y
        
        self.xstep = 1
        self.ystep = 1

        self.objX = 0
        self.objY = 14
        
        self.cicloSem = 40
        self.tiempoSem = 0
        self.colorSem = True
        self.semaforo = [23,20]

    def movexder(self):
        self.x += self.xstep
        self.grid.move_to(self, (self.y,self.x))
        self.stepAct += 1

    def movexizq(self):
        self.x -= self.xstep
        self.grid.move_to(self, (self.y,self.x))
        self.stepAct += 1

    def moveyarr(self):
        self.y -= self.ystep
        self.grid.move_to(self, (self.y,self.x))
        self.stepAct += 1
    
    def moveyab(self):
        self.y += self.ystep
        self.grid.move_to(self, (self.y,self.x))
        self.stepAct += 1

    def mover(self):
        if self.stepAct < 23:
            self.movexder()
        elif self.stepAct < 32:
            self.moveyarr()
        else:
            self.movexizq()

    def revisarSemaforo(self):
        if not(self.x == self.semaforo[1] and self.y == self.semaforo[0] and self.colorSem):
            self.mover()
        self.tiempoSem += 1
        if(self.tiempoSem >= self.cicloSem):
            self.colorSem = not(self.colorSem)
            self.tiempoSem = 0

    def printIni(self):
        print(self.x)
        print(self.y)

In [57]:
class SideWalk(ap.Agent):

    def setup(self):
        self.grid = self.model.grid
        self.group = 2

In [58]:
class CarModel(ap.Model):
    
    def setup(self):
        s = self.p.size
        n = 1
        xini = 0
        yini = 23

        self.grid = ap.Grid(self,(s,s), track_empty=True)
        self.agents = ap.AgentList(self,n,Carro,x=xini, y=yini)

        bloqueo = [(5,0),(15,0),(20,0),(30,0),(0,20),(0,30)]

        for i in range(1,40):
            if i <= 20 or i >= 30:
                bloqueo.append((5,i))
                bloqueo.append((15,i))
                bloqueo.append((20,i))
                bloqueo.append((30,i))

        for i in range(1,40):
            if i >= 30 or (i <= 20 and i >= 15) or i <= 5:
                bloqueo.append((i,20))
                bloqueo.append((i,30))    

        self.side = ap.AgentList(self,len(bloqueo), SideWalk)
        
        #self.agents[0].x = xini
        #self.agents[0].y = yini

        self.grid.add_agents(self.agents,[(yini,xini)],empty=True)
        self.grid.add_agents(self.side,bloqueo,empty=True)

    def update(self):
        
        self.x = self.agents[0].x
        self.y = self.agents[0].y

        if  self.x == self.agents[0].objX and self.y == self.agents[0].objY:
            self.stop()

    def step(self):
        self.agents.revisarSemaforo()


    def get_movement(self):
        return [self.x, self.y]

    def end(self):
        self.report('movement',self.get_movement())

In [59]:
parameters = {
    'size' : 40,
    'steps' : 100,
    'n_groups': 2
}

In [60]:
def animation_plot(model, ax):
    ap.gridplot(model.grid.attr_grid('group'), cmap='Accent', ax=ax)
    ax.set_title(f"Car Model \n Time-step: {model.t}, "
                 f"Car: {model.get_movement()}")

fig, ax = plt.subplots()
model = CarModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())